# Install Required packages

In [1]:
pip install gtts

In [2]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.8 MB/s eta 0:00:00


# Import Libraries

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from gtts import gTTS
from IPython.display import Audio
import gradio as gr

# Create APP Classses

In [4]:
class Translate_en_to_ar:
    def __init__(self,model_id="silma-ai/SILMA-9B-Instruct-v1.0"):
        # Load Model
        self.model = AutoModelForCausalLM.from_pretrained(
                    model_id,
                    device_map="auto",
                    torch_dtype=torch.bfloat16,)
        # Load Tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        # Define System Prompt
        self.system_prompt = {
            "role": "system",
            "content": (
                "أنت نموذج مختص بترجمة الأوصاف القصيرة من اللغة الإنجليزية إلى اللغة العربية الفصحى. "
                "عليك أن تنتج الترجمة العربية فقط، وتحرص على أن تكون الترجمة صحيحة ودقيقة مع بعض المرونة لتبدو طبيعية في العربية."
            )
        }
        
    def get_translation(self,text):
        # user input message
        messages = [ self.system_prompt,
            {"role": "user", "content": f"Translate the following message: '{text}'"}
        ]
        # Tokenize input
        input_ids = self.tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")
        # Generate the output
        outputs = self.model.generate(**input_ids, max_new_tokens=256)
        # Decode the output and clean it up
        output_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Make sure to output only Arabic translation
        if "model" in output_text:
            output_text = output_text.split("model")[1].strip()
        return output_text

In [5]:
class ImageCaption:
    def __init__(self,model_id="Salesforce/blip-image-captioning-large"):
        # Load model
        self.model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16).to("cuda")
        # Load Proccessor
        self.processor =  BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")

    def get_caption(self,img):
        # Get input tensor
        inputs = self.processor(img,return_tensors="pt").to("cuda", torch.float16)
        # generate output
        output = self.model.generate(**inputs)
        # decode output
        output = self.processor.decode(output[0], skip_special_tokens=True)
        return output

In [6]:
class TextToAudio:
    def __init__(self):
        pass

    def get_audio(self,text,output_file_name="output.mp3"):
        try:
            # Generate the speech using gTTS
            tts = gTTS(text=text, lang='ar')

            # Save the audio to an mp3 file
            tts.save(output_file_name)

            # Return the file path
            return output_file_name

        except Exception as e:
            print(f"Error generating audio: {e}")
            return None

# Create APP Class

In [8]:
class APP:
    def __init__(self):
        self.image_captioner  = ImageCaption()
        self.text_to_audio = TextToAudio()
        self.translator  = Translate_en_to_ar()

    def Run(self,image):
        caption = self.image_captioner.get_caption(image)
        translation = self.translator.get_translation(caption)
        audio = self.text_to_audio.get_audio(translation,'image_descripton.mp3')
        return audio

In [9]:
# create an object from APP class
app = APP()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# Create Gradio interface

In [10]:
iface = gr.Interface(
    fn=app.Run,
    inputs=gr.Image(type="numpy"),  # Accepts an image
    outputs=gr.Audio(type="filepath"),  # Outputs an audio file
    title="Image Caption Translator",
    description="Upload an image to generate an Arabic audio description.",
)

# Launch the Gradio app
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e965f38ffa3c3960b6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
